In [1]:
import string
import time
import smtplib
from getpass import getpass
import requests
from bs4 import BeautifulSoup

import helper
import json
import time
from datetime import datetime


import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [40]:
def sendEmail(data,price):
    '''
    paramter : data containing user data
    '''
    sender_email = "rimsha.maredia@gmail.com"
    recipient_email = data["email"]
    subject = "Price drop for" + data["title"]
    message = "Hello the price for " + data["title"]+ " has dropped to "+ price

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = recipient_email
    msg["Subject"] = subject

    # Attach the message to the MIMEMultipart object
    msg.attach(MIMEText(message, "plain"))

    smtp_server = "smtp.gmail.com"  # Use the SMTP server of your email provider
    smtp_port = 587  # The port may vary depending on your provider


    # Create an SMTP object and establish a connection
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.ehlo()
    server.starttls()  # Use TLS (Transport Layer Security) for encryption
    server.ehlo()
    server.login(sender_email, "ismm pcrx ygqi wmcn")  # Replace with your email password

    # Send the email
    text = msg.as_string()
    server.sendmail(sender_email, recipient_email, text)

    # Close the SMTP server connection
    server.quit()


def checkPrice(data):
    
    """
    Checks the current price and sends email if it has dropped by percentage provided by the user
    Parameters : data - dict of user data
    
    """
    
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})
    URL = data["URL"]
    print(URL)
    webpage = requests.get(URL,headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    #json_file = open("details.json", "r+")
    #json_data = json.load(json_file) 
    
    data["title"] = soup.find("span",attrs={"id":'productTitle'}).text.strip()
    
    price = float(soup.find("span", attrs={"class":'a-price'}).find("span", attrs={"class" : "a-price-whole"}).text.strip() + soup.find("span", attrs={"class":'a-price'}).find("span", attrs={"class" : "a-price-fraction"}).text.strip())
    print("Current price : " + str(price))
    
    if(data["prevprice"] == None):
        data["prevprice"] = price
    else : 
        if((price - data["prevprice"])*100/data["prevprice"] >= data["drop-percent"]):
            sendEmail(data,price)

In [41]:
def inputDetails():
    '''
    returns dictionary containing user data
    '''
    data_dict = {}
    data_dict["prevprice"] = None
    data_dict["URL"] = input("\nEnter product URL:\n")
    disc = ""
    while(True):
        try:
            disc = float(input("\nEnter the discount % required: "))
            break;
        except:
            pass
    data_dict["drop-percent"] = float(disc)
    data_dict["often"] = float(
        input("\nEnter (in secs.) how often you'd like to check the price: "))
    data_dict["email"] = input("\nEnter your Gmail ID: ")
    
    print(data_dict)
    return data_dict
        
def getTime(t):
    '''
    returns a string after converting time in seconds to hours/sec/min
    '''
    if t >= 3600:
        s = str(round(t // (3600), 2)) + " hours.\n"
    elif t >= 60:
        s = str(t // 60) + " mins, " + str(t % 60) + " secs.\n"
    else:
        s = str(t) + " secs.\n"
    return s

In [42]:
import time
from datetime import datetime

#from Tracker import inputDetails, sendEmail, checkPrice

print("~ Amazon Price Tracker ~")
data_dict = inputDetails()
iters = 0  # counts the number of price checks done
while True:
    iters += 1
    print("\nCheck #", iters, "on:", datetime.today())
    # check if the price has dropped
    try:
        checkPrice(data_dict)
    except Exception as e:
        if e != "SystemExit":
            print("\nSorry! Price tracking attempt failed.")
            exit()
    
    # print the frequency of price checks
    print("Price will be checked every " + getTime(data_dict["often"]))
    print("Interrupt keyboard to stop. (Ctrl+C)")
    time.sleep(data_dict["often"])

~ Amazon Price Tracker ~

Enter product URL:
https://www.amazon.com/gp/product/B0CDGH162F/ref=ewc_pr_img_1?smid=ATVPDKIKX0DER&psc=1

Enter the discount % required: 2

Enter (in secs.) how often you'd like to check the price: 5

Enter your Gmail ID: rimsha.maredia98@gmail.com
{'prevprice': None, 'URL': 'https://www.amazon.com/gp/product/B0CDGH162F/ref=ewc_pr_img_1?smid=ATVPDKIKX0DER&psc=1', 'drop-percent': 2.0, 'often': 5.0, 'email': 'rimsha.maredia98@gmail.com'}

Check # 1 on: 2023-11-11 11:52:32.254516
https://www.amazon.com/gp/product/B0CDGH162F/ref=ewc_pr_img_1?smid=ATVPDKIKX0DER&psc=1
Current price : 84.99
Price will be checked every 5.0 secs.

Interrupt keyboard to stop. (Ctrl+C)

Check # 2 on: 2023-11-11 11:52:40.300607
https://www.amazon.com/gp/product/B0CDGH162F/ref=ewc_pr_img_1?smid=ATVPDKIKX0DER&psc=1
Current price : 84.99
Price will be checked every 5.0 secs.

Interrupt keyboard to stop. (Ctrl+C)

Check # 3 on: 2023-11-11 11:52:48.400074
https://www.amazon.com/gp/product/B0C

KeyboardInterrupt: 